### Importing Libraries

In [3]:
import os
import numpy as np
import tensorflow as tf
import psutil

from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image

### Hardware check

In [4]:
print(f"Available CPUs: {psutil.cpu_count()}")
print(f"Available memory: {psutil.virtual_memory().available} bytes")

if tf.test.gpu_device_name():
    print("GPU found")
else:
    print("No GPU found")


physical_devices = tf.config.experimental.list_physical_devices("GPU")
print("Available GPU devices:", physical_devices)


if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Available CPUs: 8
Available memory: 2354909184 bytes
No GPU found
Available GPU devices: []


2024-05-28 14:21:57.638125: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-28 14:21:57.638285: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [1]:
def build_model(num_classes):
    base_model = MobileNetV2(
        input_shape=(224, 224, 3), include_top=False, weights="imagenet"
    )

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation="relu")(x)
    predictions = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


num_classes = 2 
model = build_model(num_classes)
model.summary()

NameError: name 'MobileNetV2' is not defined

### Loading and Preparing Training Data

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

train_generator = train_datagen.flow_from_directory(
    "output_batches/train", 
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
)

### Training the Model with Checkpoints

In [ ]:

checkpoint_path = "checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    period=1, 
)


model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


history = model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    callbacks=[cp_callback],
    verbose=1,
)


model.save("bird_detection_model.h5")